#### Notebook Imports

In [ ]:
%%configure
{
    "defaultLakehouse": {  
        "name": "sample_lakehouse",
    }
}
# Comment this out if not needed.

In [ ]:
%run evaluation_utils

# Agent Configs

## Datasource Configs

### Projects Lakehouse

In [ ]:
# Datasource
datasource_name = 'datasource_name'
datasource_type = 'lakehouse, warehouse, kqldatabase, semanticmodel'
datasource_description = """
Datasource description
"""
datasource_instructions = """
Datasource instructions
"""
datasource_tables = ['sample_table']
datasource_eval_tables = ['sample_table_snapshot']
datasource_examples = {
    "Example question 1": "SELECT * AS example_column FROM dbo.sample_table;",
    "Example question 2": "SELECT COUNT(*) AS example_count FROM dbo.sample_table WHERE condition = 'value';"
}

In [ ]:
create_update_datasource(
    datasourceName = datasource_name,
    datasourceType = datasource_type,
    datasourceDescription = datasource_description,
    datasourceInstructions = datasource_instructions,
    datasourceTables = datasource_tables,
    datasourceEvalTables = datasource_eval_tables,
    datasourceExamples = datasource_examples
)

## Agent Config

In [ ]:
# Agent
agent_name = 'sample_agent_name'
agent_datasources = ['datasource_name'] # remember to add version number
agent_instructions = """
Agent instructions
"""
agent_evals = [
    # Sample Eval Pairs
    {"question": "Example question 1", "expected_answer": "Expected answer 1."},
    {"question": "Example question 2", "expected_answer": "Expected answer 2."}
]

# Agent Creation

In [ ]:
# Importing tools from Fabric Data Agent SDK
from fabric.dataagent.client import (
    FabricDataAgentManagement,
    create_data_agent,
    delete_data_agent,
)

In [ ]:
# Create Data Agent
data_agent = create_data_agent(agent_name)
data_agent = FabricDataAgentManagement(agent_name)

In [ ]:
# Assign Agent Instructions
data_agent.update_configuration(
    instructions= agent_instructions,
)

In [ ]:
# Run this if you developing within the Fabric Portal and can actively retrieve the list of data sources available in the workspace
def safe_get_datasources(): # add_datasource calls get_datasources, which doesn't handle None types.... This is a modified function which handles this
    config = data_agent._client.get_configuration()
    data_sources = config.value.get("dataSources") or []
    return [data_agent._client.get_datasource(ds["id"]) for ds in data_sources]

data_agent._client.get_datasources = safe_get_datasources

## Configuring Data Source Instructions, Description and Example Queries

In [ ]:
# Adding all lakehouse data sources
datasource_count = 0
for datasource_key in agent_datasources:
    # Adding data source to agent
    data_agent.add_datasource(get_name(datasource_key), type=get_type(datasource_key))
    datasource = data_agent.get_datasources()[datasource_count]

    # Adding tables to agent datasource
    for table_name in  get_tables(datasource_key):
        datasource.select("dbo", table_name)

    # Adding Instructions and Description to datasource
    datasource.update_configuration(instructions=get_instructions(datasource_key), user_description = get_description(datasource_key))

    # Adding example question query pairs to datasource
    datasource.add_fewshots(get_examples(datasource_key))

    datasource_count += 1

### Log and Publish Agent

In [ ]:
# Creating Row in Agents Table and Publish Agent
create_update_agent(
    agentName=agent_name,
    agentInstructions=agent_instructions,
    agentDatasources=agent_datasources,
    agentEvals=agent_evals
)

data_agent.publish()
# data_agent.delete()

# Agent Evaluation

In [ ]:
workspace_name = "sample_workspace_name"
eval_id = evaluate_agent(agent_name, workspace_name)